<a href="https://colab.research.google.com/github/DanielGirald/Senalesysistemas/blob/main/corte_2/Proyecto/Proyecto_Sys_2025_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
# ---------------------------------------------
# 📦 Instalación de librerías necesarias
# ---------------------------------------------
!pip install streamlit
!pip install streamlit-option-menu
!pip install pyngrok  # No lo usaremos, pero útil si decides cambiar en el futuro

# Descargar Cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

In [108]:
%%writefile app.py
import streamlit as st

# Configuración general del dashboard
st.set_page_config(
    page_title="🔊 Proyecto Final - Comunicación Digital",
    layout="wide"
)

# Menú horizontal con st.radio
secciones = st.radio(
    "📑 Selecciona una sección para explorar:",
    [
        "🏠 Inicio",
        "📊 Fase 1: Señal compuesta",
        "📡 Fase 2: Señales I/Q",
        "🧩 Fase 3: Modulación QAM-16",
        "🌐 Fase 4: OFDM + Ruido",
        "🎯 Resultado Final"
    ],
    horizontal=True
)

# Contenido para la sección Inicio
if secciones == "🏠 Inicio":
    st.markdown("## 📡 Proyecto Final - Comunicaciones Digitales Avanzadas")

    with st.container():
        col1, col2 = st.columns([2, 1])
        with col1:
            st.write(
                """
                Este proyecto permite simular paso a paso un sistema de comunicaciones digitales moderno.
                Podrás analizar señales, aplicar modulación QAM, simular transmisión OFDM y estudiar el efecto del ruido AWGN.

                ### 🎓 Integrantes del grupo:
                - **Daniel David Giraldo Clavijo** (1006552543)
                - **Freyder Estiven Girldo Vivas** (1193089612)
                """
            )
        with col2:
            st.success("¡Bienvenido! Usa el menú de arriba para navegar 🔼")

    st.info("💡 Consejo: Explora cada fase en orden para comprender el flujo completo del sistema.")

# El resto de las fases se definirán en otras celdas


Overwriting app.py


In [109]:
%%writefile -a app.py
# -----------------------------------------
# 📡 Fase 1: Señal compuesta y análisis espectral
# -----------------------------------------
elif secciones == "📊 Fase 1: Señal compuesta":

    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.signal import butter, lfilter
    from scipy.fftpack import fft
    import pandas as pd

    st.header("🎛️ Análisis espectral de una señal compuesta")
    st.markdown("Ajusta las frecuencias y observa cómo cambia la señal en el tiempo y en el espectro.")

    with st.expander("🎚️ Parámetros de entrada", expanded=True):
        col1, col2 = st.columns(2)
        with col1:
            f1 = st.slider("Frecuencia 1 (Hz)", 10, 100, 50)
            f2 = st.slider("Frecuencia 2 (Hz)", 100, 300, 120)
        with col2:
            f3 = st.slider("Frecuencia 3 (Hz)", 200, 500, 300)
            fcorte = st.slider("Frecuencia de corte del filtro (Hz)", 50, 400, 150)

    fs = 1000
    t = np.linspace(0, 1, fs, endpoint=False)

    # Generar señal compuesta
    x = (
        np.sin(2 * np.pi * f1 * t) +
        0.5 * np.sin(2 * np.pi * f2 * t) +
        0.2 * np.sin(2 * np.pi * f3 * t)
    )

    # Filtro pasa-bajo Butterworth
    order = 4
    b, a = butter(order, fcorte / (0.5 * fs), btype='low')
    y = lfilter(b, a, x)

    # Visualización
    tab1, tab2 = st.tabs(["🕒 Dominio del tiempo", "📈 Dominio de la frecuencia"])

    with tab1:
        fig, ax = plt.subplots()
        ax.plot(t, x, label="Señal Original")
        ax.plot(t, y, label="Filtrada", linestyle='--')
        ax.set_title("Señal en el tiempo")
        ax.set_xlabel("Tiempo [s]")
        ax.set_ylabel("Amplitud")
        ax.legend()
        st.pyplot(fig)

    with tab2:
        freqs = np.fft.fftfreq(len(t), 1/fs)
        X = np.abs(fft(x))
        Y = np.abs(fft(y))

        fig2, ax2 = plt.subplots()
        ax2.plot(freqs[:fs//2], X[:fs//2], label="Original")
        ax2.plot(freqs[:fs//2], Y[:fs//2], label="Filtrada", linestyle='--')
        ax2.set_title("Espectro de Frecuencia")
        ax2.set_xlabel("Frecuencia [Hz]")
        ax2.set_ylabel("Magnitud")
        ax2.legend()
        st.pyplot(fig2)

    st.success("🎉 Fase 1 completada: ¡observa cómo las frecuencias afectan la señal!")


Appending to app.py


In [110]:
%%writefile -a app.py
# -----------------------------------------------------------
# 🧠 FASE 2: Construcción de Señales I/Q con Transformada de Hilbert
# -----------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert

# Página de Fase 2
if secciones == "📡 Fase 2: Señales I/Q":

    st.header("🧠 Fase 2: Construcción de Señales I/Q mediante Transformada de Hilbert")
    st.markdown("""
    Esta fase genera una señal **I(t)** y, usando la **Transformada de Hilbert**, construye su componente en cuadratura **Q(t)**.
    También se muestra el **módulo de la señal compleja** y su análisis espectral con FFT.
    """)

    with st.expander("🎚️ Parámetros ajustables", expanded=True):
        col1, col2, col3 = st.columns(3)
        with col1:
            f0 = st.slider("🎵 Frecuencia de I(t) [Hz]", 1, 50, 10)
        with col2:
            fs = st.slider("🧪 Frecuencia de muestreo [Hz]", 200, 2000, 1000)
        with col3:
            duracion = st.slider("⏱️ Duración de la señal [s]", 1, 5, 1)

    # Tiempo
    t = np.linspace(0, duracion, int(fs * duracion), endpoint=False)

    # Señal I(t)
    I_t = np.sin(2 * np.pi * f0 * t)

    # Transformada de Hilbert para obtener Q(t)
    x_a = hilbert(I_t)
    Q_t = np.imag(x_a)
    modulo = np.abs(x_a)

    # Paleta nueva
    c_I = "#00B5AD"      # teal
    c_Q = "#D81B60"      # magenta
    c_mod = "#F9A825"    # ámbar
    c_I_fft = "#3F51B5"  # índigo
    c_Q_fft = "#E91E63"  # fucsia
    c_mod_fft = "#009688" # teal oscuro

    # Tabs para organizar visualizaciones
    tab1, tab2 = st.tabs(["📈 Señales en el tiempo", "🔍 Análisis espectral"])

    with tab1:
        st.subheader("🟦 Señales I(t), Q(t) y módulo en el tiempo")
        fig1, ax1 = plt.subplots(figsize=(10, 4))
        ax1.plot(t, I_t, label="I(t) - Parte real", color=c_I)
        ax1.plot(t, Q_t, label="Q(t) - Parte imaginaria", color=c_Q)
        ax1.plot(t, modulo, '--', label="Módulo", color=c_mod)
        ax1.set_xlabel("Tiempo [s]")
        ax1.set_ylabel("Amplitud")
        ax1.set_title("Señales I(t), Q(t) y Módulo")
        ax1.legend()
        ax1.grid(True)
        st.pyplot(fig1)

    with tab2:
        st.subheader("🔬 FFT de las señales I(t), Q(t) y módulo")
        N = len(t)
        freqs = np.fft.fftfreq(N, 1 / fs)
        freqs_shifted = np.fft.fftshift(freqs)

        X_I = np.fft.fft(I_t) / N
        X_Q = np.fft.fft(Q_t) / N
        X_mod = np.fft.fft(modulo) / N

        X_I_mag = 2 * np.abs(np.fft.fftshift(X_I))
        X_Q_mag = 2 * np.abs(np.fft.fftshift(X_Q))
        X_mod_mag = 2 * np.abs(np.fft.fftshift(X_mod))

        fig2, ax2 = plt.subplots(figsize=(10, 4))
        ax2.plot(freqs_shifted, X_I_mag, label='FFT I(t)', color=c_I_fft)
        ax2.plot(freqs_shifted, X_Q_mag, label='FFT Q(t)', color=c_Q_fft)
        ax2.plot(freqs_shifted, X_mod_mag, label='FFT módulo', color=c_mod_fft)
        ax2.set_xlim(-100, 300)
        ax2.set_xlabel("Frecuencia [Hz]")
        ax2.set_ylabel("Amplitud")
        ax2.set_title("Espectro de las señales I/Q y módulo")
        ax2.legend()
        ax2.grid(True)
        st.pyplot(fig2)

    st.success("✅ Fase 2 completada: ¡las señales I/Q y su espectro han sido visualizados con éxito!")




Appending to app.py


In [111]:
%%writefile -a app.py
# ---------------------------------------------
# 🧩 Fase 3 – Modulación QAM-16 y Banda Pasante
# ---------------------------------------------
elif secciones == "🧩 Fase 3: Modulación QAM-16":

    import numpy as np
    import matplotlib.pyplot as plt

    st.header("🧩 Fase 3: Modulación QAM-16 y Señal en Banda Pasante")
    st.markdown("""
    En esta fase simulamos una modulación **16-QAM**, construimos su diagrama de constelación y generamos una señal modulada en banda pasante.

    La señal está normalizada para asegurar una potencia promedio unitaria.
    """)

    with st.expander("🎚️ Parámetros de simulación", expanded=True):
        col1, col2, col3 = st.columns(3)
        with col1:
            num_symbols = st.slider("🔢 Cantidad de símbolos", 100, 1000, 400, step=100)
        with col2:
            fc = st.slider("📡 Frecuencia de portadora (Hz)", 500, 5000, 1000, step=500)
        with col3:
            fs = st.slider("🧪 Frecuencia de muestreo (Hz)", 5000, 50000, 10000, step=5000)

    # Parámetros de modulación
    M = 16
    k = int(np.log2(M))
    bits = np.random.randint(0, 2, num_symbols * k)
    symbols = bits.reshape((-1, k))

    # Mapeo I/Q
    mapping = {
        (0,0,0,0): (-3,-3), (0,0,0,1): (-3,-1), (0,0,1,1): (-3, 1), (0,0,1,0): (-3, 3),
        (0,1,0,0): (-1,-3), (0,1,0,1): (-1,-1), (0,1,1,1): (-1, 1), (0,1,1,0): (-1, 3),
        (1,1,0,0): ( 1,-3), (1,1,0,1): ( 1,-1), (1,1,1,1): ( 1, 1), (1,1,1,0): ( 1, 3),
        (1,0,0,0): ( 3,-3), (1,0,0,1): ( 3,-1), (1,0,1,1): ( 3, 1), (1,0,1,0): ( 3, 3)
    }

    I, Q = [], []
    for b in symbols:
        i, q = mapping[tuple(b)]
        I.append(i)
        Q.append(q)

    I = np.array(I) / np.sqrt(10)
    Q = np.array(Q) / np.sqrt(10)

    # Nuevos colores
    color_constelacion = '#00CED1'  # turquesa
    color_portadora = '#FF8C00'     # naranja oscuro

    tab1, tab2 = st.tabs(["📍 Diagrama de Constelación", "📶 Señal Modulada"])

    with tab1:
        st.subheader("🎯 Diagrama de Constelación Normalizada")
        fig1, ax1 = plt.subplots()
        ax1.scatter(I, Q, color=color_constelacion, alpha=0.7, edgecolors="black")
        ax1.axhline(0, color='gray', linewidth=0.5)
        ax1.axvline(0, color='gray', linewidth=0.5)
        ax1.set_xlabel("I (In-Phase)")
        ax1.set_ylabel("Q (Quadrature)")
        ax1.set_title("📍 Constelación 16-QAM")
        ax1.set_aspect("equal")
        ax1.grid(True)
        st.pyplot(fig1)

    with tab2:
        st.subheader("📡 Señal en Banda Pasante")

        samples_per_symbol = int(fs / fc)
        t_symbol = np.linspace(0, 1/fs * samples_per_symbol, samples_per_symbol, endpoint=False)
        s = []

        for i_sym, q_sym in zip(I, Q):
            st_sample = i_sym * np.cos(2*np.pi*fc*t_symbol) - q_sym * np.sin(2*np.pi*fc*t_symbol)
            s.extend(st_sample)

        s = np.array(s)

        duration_to_plot = st.slider("🕒 Duración a visualizar [ms]", 1, 10, 5)
        N = int(fs * (duration_to_plot / 1000))
        t_total = np.linspace(0, N/fs, N)

        fig2, ax2 = plt.subplots()
        ax2.plot(t_total, s[:N], color=color_portadora)
        ax2.set_xlabel("Tiempo [s]")
        ax2.set_ylabel("Amplitud")
        ax2.set_title(f"🧪 Fragmento de señal modulada ({duration_to_plot} ms)")
        ax2.grid(True)
        st.pyplot(fig2)

    st.success("✅ Fase 3 completada con nuevos colores aplicados.")




Appending to app.py


In [112]:
%%writefile -a app.py
# ---------------------------------------------------------------
# 🌐 Fase 4 - Canal con ruido AWGN y análisis de constelación
# ---------------------------------------------------------------
elif secciones == "🌐 Fase 4: OFDM + Ruido":

    import numpy as np
    import matplotlib.pyplot as plt

    st.header("🌐 Fase 4: Transmisión con Ruido y Recepción de la Señal")
    st.markdown("""
    En esta fase simulamos el efecto del **ruido blanco aditivo gaussiano (AWGN)** sobre una señal modulada QAM-16.

    El ruido se ajusta mediante la relación señal/ruido (**SNR**) para observar cómo afecta a la constelación recibida.
    """)

    with st.expander("🎚️ Parámetro del canal", expanded=True):
        snr_db = st.slider("📶 Relación Señal/Ruido (SNR) [dB]", 0, 40, 20, step=5)

    M = 16
    k = int(np.log2(M))
    bits = np.random.randint(0, 2, 1000 * k)
    symbols = bits.reshape(-1, k)

    # Mapeo QAM-16
    mapping = {
        (0,0,0,0): (-3,-3), (0,0,0,1): (-3,-1), (0,0,1,1): (-3, 1), (0,0,1,0): (-3, 3),
        (0,1,0,0): (-1,-3), (0,1,0,1): (-1,-1), (0,1,1,1): (-1, 1), (0,1,1,0): (-1, 3),
        (1,1,0,0): ( 1,-3), (1,1,0,1): ( 1,-1), (1,1,1,1): ( 1, 1), (1,1,1,0): ( 1, 3),
        (1,0,0,0): ( 3,-3), (1,0,0,1): ( 3,-1), (1,0,1,1): ( 3, 1), (1,0,1,0): ( 3, 3)
    }

    iq_symbols = np.array([mapping[tuple(b)] for b in symbols])
    I = iq_symbols[:, 0]
    Q = iq_symbols[:, 1]

    # Transmisión
    tx_signal = I + 1j * Q

    # Calcular potencia promedio y ruido
    snr_linear = 10 ** (snr_db / 10)
    power_signal = np.mean(np.abs(tx_signal)**2)
    noise_power = power_signal / snr_linear
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*tx_signal.shape) + 1j * np.random.randn(*tx_signal.shape))

    # Recepción
    rx_signal = tx_signal + noise

    # Separar I y Q
    rx_I = np.real(rx_signal)
    rx_Q = np.imag(rx_signal)

    st.subheader("📍 Diagrama de Constelación con Ruido")
    fig, ax = plt.subplots()
    ax.scatter(rx_I, rx_Q, c="purple", alpha=0.5, edgecolors="black", s=20)
    ax.set_title(f"Constelación QAM-16 con SNR = {snr_db} dB")
    ax.set_xlabel("I(t)")
    ax.set_ylabel("Q(t)")
    ax.grid(True)
    st.pyplot(fig)

    st.info("💡 Entre menor sea el SNR, más dispersos estarán los puntos.")
    st.success("✅ Fase 4 completada: ¡observaste el efecto del ruido en la señal digital!")


Appending to app.py


In [113]:
%%writefile -a app.py
# ------------------------------------------
# 🎯 FASE FINAL - Resultado del sistema completo
# ------------------------------------------
elif secciones == "🎯 Resultado Final":

    import numpy as np
    import matplotlib.pyplot as plt

    st.header("🎯 Resultado Final: Transmisión Completa con QAM-16")
    st.markdown("""
    Esta fase muestra el resultado final del sistema completo de comunicación digital,
    desde la generación de símbolos hasta su recepción con ruido (**AWGN**).

    Podrás ajustar el nivel de ruido (**SNR**) y observar cómo afecta la constelación.
    """)

    with st.expander("🎚️ Parámetro de ruido (canal AWGN)", expanded=True):
        snr_db_final = st.slider("🔊 Nivel de ruido SNR [dB]", 5, 40, 20, step=1)

    # Parámetros
    M = 16
    k = int(np.log2(M))
    bits = np.random.randint(0, 2, 1000 * k)
    symbols = bits.reshape((-1, k))

    # Mapeo QAM-16
    mapping = {
        (0,0,0,0): (-3,-3), (0,0,0,1): (-3,-1), (0,0,1,1): (-3, 1), (0,0,1,0): (-3, 3),
        (0,1,0,0): (-1,-3), (0,1,0,1): (-1,-1), (0,1,1,1): (-1, 1), (0,1,1,0): (-1, 3),
        (1,1,0,0): ( 1,-3), (1,1,0,1): ( 1,-1), (1,1,1,1): ( 1, 1), (1,1,1,0): ( 1, 3),
        (1,0,0,0): ( 3,-3), (1,0,0,1): ( 3,-1), (1,0,1,1): ( 3, 1), (1,0,1,0): ( 3, 3)
    }

    iq_symbols = np.array([mapping[tuple(b)] for b in symbols])
    I = iq_symbols[:, 0] / np.sqrt(10)
    Q = iq_symbols[:, 1] / np.sqrt(10)
    tx_signal = I + 1j * Q

    # Ruido
    snr_linear = 10 ** (snr_db_final / 10)
    power_signal = np.mean(np.abs(tx_signal)**2)
    noise_power = power_signal / snr_linear
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*tx_signal.shape) + 1j * np.random.randn(*tx_signal.shape))

    rx_signal = tx_signal + noise
    rx_I = np.real(rx_signal)
    rx_Q = np.imag(rx_signal)

    # Nuevos colores
    color_tx = '#1ABC9C'  # verde agua
    color_rx = '#E74C3C'  # rojo coral

    # Tabs corregidas
    tab1, tab2 = st.tabs(["📍 Constelación transmitida", "📡 Constelación recibida con ruido"])

    with tab1:
        st.subheader("📦 Constelación ideal (sin ruido)")
        fig1, ax1 = plt.subplots()
        ax1.scatter(I, Q, c=color_tx, alpha=0.7, edgecolors='black')
        ax1.set_xlabel("I(t)")
        ax1.set_ylabel("Q(t)")
        ax1.set_title("Constelación transmitida QAM-16")
        ax1.set_aspect("equal")
        ax1.grid(True)
        st.pyplot(fig1)

    with tab2:
        st.subheader(f"📡 Señal recibida con SNR = {snr_db_final} dB")
        fig2, ax2 = plt.subplots()
        ax2.scatter(rx_I, rx_Q, c=color_rx, alpha=0.5, edgecolors='black')
        ax2.set_xlabel("I(t)")
        ax2.set_ylabel("Q(t)")
        ax2.set_title("Constelación recibida con ruido")
        ax2.set_aspect("equal")
        ax2.grid(True)
        st.pyplot(fig2)

    st.info("🔍 Observa cómo el ruido afecta la claridad de la constelación.")
    st.success("🚀 Simulación final completada con éxito.")

        # Conclusión visual del proyecto
    st.markdown("""
    ---
    ### 🎯 **Finalidad del Proyecto**

    Esta simulación muestra cómo se **codifican, modulan y transmiten datos digitales** en sistemas de comunicación modernos.

    - 📡 **QAM-16** permite enviar 4 bits por símbolo (**alta eficiencia espectral**).
    - 🔄 **OFDM** mejora la robustez en canales multipath (como en WiFi y 5G).
    - 📊 **Ruido AWGN** simula un canal real y permite estudiar la calidad de recepción.

    ✅ Este tipo de procesamiento es el **núcleo de tecnologías inalámbricas modernas** como:

    - WiFi 802.11ac/ax
    - 5G NR (New Radio)
    - LTE (4G)
    """, unsafe_allow_html=True)



Appending to app.py


In [114]:
import os
import threading
import time

# ✅ Iniciar Streamlit en segundo plano
def run_streamlit():
    os.system("streamlit run app.py &")

# 🧵 Lanzar Streamlit en hilo separado
threading.Thread(target=run_streamlit).start()

# ⏱️ Esperar que arranque
time.sleep(5)

# 🌐 Ejecutar túnel Cloudflared
print("🌐 Iniciando Cloudflared...")
print("⌛ Espera unos segundos mientras se genera el enlace...")
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate


🌐 Iniciando Cloudflared...
⌛ Espera unos segundos mientras se genera el enlace...
2025-07-25T02:25:52Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-07-25T02:25:52Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-07-25T02:25:56Z INF +--------------------------------------------------------------------------------------------+
2025-07-25T02:25:56Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |